# Importowanie bibliotek



In [2]:
import pandas as pd
import matplotlib
import glob
import os
import openpyxl
import time

# Wczytanie pliku CSV jako tabeli

In [3]:
df=pd.DataFrame()
#Wczytanie pliku CSV z komputera
path = 'C:/Users/barto/Desktop/Praca magisterska - dane/CSV/*.csv'
support_file_path = 'C:/Users/barto/Desktop/Praca magisterska - dane/Support.csv'
file_names = glob.glob(path)
data_frames = []
start_time = time.time()
for file_name in file_names:
    df = pd.read_csv(file_name, encoding = 'unicode_escape', sep = ";", header = None, error_bad_lines = False, engine = 'python', usecols = [0, 5, 7, 8, 12, 13, 14, 15, 17, 22])
    data_frames.append(df)
df = pd.concat(data_frames, ignore_index=True)
#Nazwanie kolumn wg standardu plików CSV z ZTM'u
df.columns=['DT_KARTY', 'LP_KURSU', 'NR_LINI', 'LP_PRZYST', 'RJ_KIEDY', 'DATA', 'KIEDY', 'POSTOJ', 'SYM_SLUPKA', 'OPIS_TABL']
#Usunięcie linii autobusowych
df3 = df[~df['NR_LINI'].str.contains("[a-zA-Z]").fillna(False)]
df3['NR_LINI']=df3['NR_LINI'].astype(int)
df3.drop(df3[df3['NR_LINI'] > 100].index, inplace = True)
#Usunięcie NaN
df4=df3.dropna(subset=['DATA'])
#Usunięcie komórek PUSTA
df4.drop(df4[df4['OPIS_TABL'] == 'PUSTA'].index, inplace = True)
#Usunięcie postojów dłuższych niż 180 sekund
df4.drop(df4[df4['POSTOJ'] > 180].index, inplace = True)
df4.reset_index(drop = True, inplace = True)
df4.to_csv(support_file_path, index = False)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: {:.2f} seconds".format(elapsed_time))

C:\Users\barto\AppData\Local\Temp\ipykernel_12020\235634565.py:9: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(file_name, encoding = 'unicode_escape', sep = ";", header = None, error_bad_lines = False, engine = 'python', usecols = [0, 5, 7, 8, 12, 13, 14, 15, 17, 22])
C:\Users\barto\AppData\Local\Temp\ipykernel_12020\235634565.py:9: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(file_name, encoding = 'unicode_escape', sep = ";", header = None, error_bad_lines = False, engine = 'python', usecols = [0, 5, 7, 8, 12, 13, 14, 15, 17, 22])
C:\Users\barto\AppData\Local\Temp\ipykernel_12020\235634565.py:9: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(file_name, e

Elapsed time: 54.57 seconds


In [4]:
df4

,DT_KARTY,LP_KURSU,NR_LINI,LP_PRZYST,RJ_KIEDY,DATA,KIEDY,POSTOJ,SYM_SLUPKA,OPIS_TABL
0,2023-01-01,3,1,1,11:41,2023-01-01,11:41:28,22,STAR02,Junikowo
1,2023-01-01,3,1,2,11:42,2023-01-01,11:42:30,10,BYST02,Junikowo
2,2023-01-01,3,1,3,11:43,2023-01-01,11:43:54,12,STOM02,Junikowo
3,2023-01-01,3,1,4,11:45,2023-01-01,11:46:04,64,RSTA06,Junikowo
4,2023-01-01,3,1,5,11:46,2023-01-01,11:47:14,14,RSTA03,Junikowo
...,...,...,...,...,...,...,...,...,...,...
876200,2023-01-15,70,98,23,23:24,2023-01-15,23:24:02,0,P K01,Zjazd /Zajezdnia Franowo
876201,2023-01-15,70,98,24,23:25,2023-01-15,23:24:34,2,SZWE01,Zjazd /Zajezdnia Franowo
876202,2023-01-15,70,98,25,23:26,2023-01-15,23:25:42,4,SZWA01,Zjazd /Zajezdnia Franowo
876203,2023-01-15,70,98,26,23:27,2023-01-15,23:26:22,0,FRWO01,Zjazd /Zajezdnia Franowo


# Wypisanie par przystanków

In [5]:
def zmiana(U8):
    i = 0
    sekundy = 0
    for element in U8:
        if element == ':' : sekundy += 0
        elif i == 0 : sekundy += 36000 * int(element)
        elif i == 1 : sekundy += 3600 * int(element)
        elif i == 3 : sekundy += 600 * int(element)
        elif i == 4 : sekundy += 60 * int(element)
        elif i == 6 : sekundy += 10 * int(element)
        elif i == 7 : sekundy += int(element)
        else : break
        i += 1
    return sekundy

In [6]:
#Stworzenie ramki danych 
przystanki = pd.DataFrame({"PRZYSTANEK 1":[0], "PRZYSTANEK 2":[0], "ŚREDNI CZAS":[0], "ROZKŁADOWY CZAS":[0], "NAZWA":[0], "SUMA PRZEJAZDÓW":[0], "ILOŚĆ PRZEJAZDÓW":[0], "KTÓRE LINIE":[0]})
przystanki["KTÓRE LINIE"] = przystanki["KTÓRE LINIE"].astype(str)
chunk_size = 2500
count = 0
lp_przyst = 1
index_pary = 0
(num_rows, num_col) = df4.shape
numer = 0
start_time = time.time()
iterator = pd.read_csv(support_file_path, chunksize = chunk_size, low_memory = False, iterator = True)
for chunk in iterator :
    (chunk_rows, chunk_col) = chunk.shape
    chunk_index = -1
    for index, row in chunk.iterrows():
        chunk_index += 1
        if chunk_index + 2 > chunk_rows : continue
        elif index + 2 > num_rows : continue
        else :
            przyst1 = chunk.loc[index, 'SYM_SLUPKA']
            przyst2 = chunk.loc[index + 1, 'SYM_SLUPKA']
            #obliczenie czasu przejazdu
            przejazd = zmiana(chunk.loc[index + 1, 'KIEDY']) - chunk.loc[index + 1, 'POSTOJ'] - zmiana(chunk.loc[index, 'KIEDY'])
            #obliczenie rozkładowego czasu przejazdu
            przejazd_roz = zmiana(chunk.loc[index + 1, "RJ_KIEDY"]) - zmiana(chunk.loc[index, "RJ_KIEDY"])
            nazwa = "{}{}".format(przyst1, przyst2)
            liczba_przyst1 = chunk.loc[index, 'LP_PRZYST']
            liczba_przyst2 = chunk.loc[index+1, 'LP_PRZYST']
            liczba_kursu1 = chunk.loc[index,'LP_KURSU']
            liczba_kursu2 = chunk.loc[index + 1, 'LP_KURSU']
            nr_linii = chunk.loc[index, 'NR_LINI']
            if (przejazd < 0) : continue
            elif (przejazd > (7*60)): continue
            #warunek wykrywający
            elif (liczba_przyst1 == lp_przyst and liczba_przyst2 == lp_przyst + 1) and (liczba_kursu1 == liczba_kursu2):
                if (przystanki["NAZWA"] == nazwa).any() :
                    i = przystanki[przystanki["NAZWA"] == nazwa].index[0]
                    przystanki.loc[i, "SUMA PRZEJAZDÓW"] += przejazd
                    przystanki.loc[i, "ILOŚĆ PRZEJAZDÓW"] += 1
                    linie = przystanki.loc[i, "KTÓRE LINIE"].split(",")
                    nowa_linia = True
                    for subarray in linie:
                        if subarray == str(nr_linii):
                            nowa_linia = False
                    if nowa_linia :
                        przystanki.loc[i, "KTÓRE LINIE"] += str(nr_linii) + ','
                else:
                    przystanki.loc[numer, "PRZYSTANEK 1"] = przyst1
                    przystanki.loc[numer, "PRZYSTANEK 2"] = przyst2
                    przystanki.loc[numer, "ŚREDNI CZAS"] = 0
                    przystanki.loc[numer, "ROZKŁADOWY CZAS"] = przejazd_roz
                    przystanki.loc[numer, "NAZWA"] = nazwa
                    przystanki.loc[numer, "SUMA PRZEJAZDÓW"] = 0
                    przystanki.loc[numer, "ILOŚĆ PRZEJAZDÓW"] = 0
                    przystanki.loc[numer, "KTÓRE LINIE"] = 0
                    przystanki.loc[numer, "SUMA PRZEJAZDÓW"] += przejazd
                    przystanki.loc[numer, "ILOŚĆ PRZEJAZDÓW"] += 1
                    przystanki.loc[numer, "KTÓRE LINIE"] = str(nr_linii) + ","
                    numer += 1
                lp_przyst += 1
                index_pary += 1
            else:
                lp_przyst = 1
    pass
przystanki["ŚREDNI CZAS"] = przystanki["SUMA PRZEJAZDÓW"]/przystanki["ILOŚĆ PRZEJAZDÓW"]
przystanki["ŚREDNI CZAS"] = przystanki["ŚREDNI CZAS"].round(0)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: {:.2f} seconds".format(elapsed_time))        

Elapsed time: 435.58 seconds


Średni czas przejazdu dla linii/wagonu.


In [7]:
przystanki


,PRZYSTANEK 1,PRZYSTANEK 2,ŚREDNI CZAS,ROZKŁADOWY CZAS,NAZWA,SUMA PRZEJAZDÓW,ILOŚĆ PRZEJAZDÓW,KTÓRE LINIE
0,STAR02,BYST02,56.0,60.0,STAR02BYST02,55636.0,994.0,"1,11,12,13,18,5,6,7,8,14,"
1,BYST02,STOM02,61.0,60.0,BYST02STOM02,60634.0,998.0,"1,11,12,13,18,5,6,7,8,14,"
2,STOM02,RSTA06,65.0,120.0,STOM02RSTA06,64470.0,997.0,"1,11,12,13,18,5,6,7,8,14,"
3,RSTA06,RSTA03,47.0,60.0,RSTA06RSTA03,8606.0,184.0,"1,11,13,5,7,8,14,"
4,RSTA03,HEWI01,72.0,60.0,RSTA03HEWI01,86116.0,1195.0,"1,11,13,14,3,5,6,7,8,10,"
...,...,...,...,...,...,...,...,...
326,KRAU92,S-1G,266.0,60.0,KRAU92S-1G,7450.0,28.0,"14,5,8,"
327,AREN01,RJNJ02,148.0,180.0,AREN01RJNJ02,6350.0,43.0,"15,6,"
328,GLHE05,S-1G,293.0,120.0,GLHE05S-1G,4394.0,15.0,"15,6,7,"
329,PAMI91,KOLE01,320.0,180.0,PAMI91KOLE01,640.0,2.0,"9,"


Wstęp teoretyczny:
- smart city


# Propozycje:
- usunięcie kursów zjazdowych

# Dalsze prace:
- stworzenie modelu do optymalizacji
- uwzględnienie rozkładów minutowych

# Generowanie mapy wizualnej 


In [8]:
slupeczki = przystanki.drop(["ŚREDNI CZAS","ROZKŁADOWY CZAS", "NAZWA", "SUMA PRZEJAZDÓW", "ILOŚĆ PRZEJAZDÓW", "KTÓRE LINIE"], axis = 1)

In [9]:
slupki_przystankow = pd.DataFrame({'NAZWA':[0], 'E':[0], 'N':[0]})
(num_rows, num_col) = przystanki.shape
numer = 0
start_time = time.time()
slupeczki = przystanki.drop(["ŚREDNI CZAS", "NAZWA", "SUMA PRZEJAZDÓW", "ILOŚĆ PRZEJAZDÓW", "KTÓRE LINIE"], axis = 1)
for index, row in slupeczki.iterrows():
    nazwa = "{}".format(slupeczki.loc[index, "PRZYSTANEK 1"])
    if index + 1 > num_rows : continue
    elif (slupki_przystankow['NAZWA'] == nazwa).any() :
        continue
    else:
        slupki_przystankow.loc[numer, 'NAZWA'] = nazwa
        numer += 1
       

end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: {:.2f} seconds".format(elapsed_time))

Elapsed time: 0.15 seconds


In [10]:
koordy = pd.DataFrame()
koordy_path = r"C:/Users/barto/Desktop/Praca magisterska - dane/słupki BusMan 23.01.2023.xlsx"
koordy = pd.read_excel(koordy_path, usecols = [1, 3, 4])
koordy = koordy.dropna()
for index, row in slupki_przystankow.iterrows():
    nazwa = slupki_przystankow.loc[index, "NAZWA"]
    if (koordy["Symbol obecny"] == nazwa).any() :
        i = koordy[koordy["Symbol obecny"] == nazwa].index[0]
        slupki_przystankow.loc[index, "E"] = koordy.loc[i, "gps E"]
        slupki_przystankow.loc[index, "N"] = koordy.loc[i, "gps N"]
    

In [11]:
slupki_przystankow

,NAZWA,E,N
0,STAR02,16.932725,52.368768
1,BYST02,16.934045,52.370723
2,STOM02,NaN,NaN
3,RSTA06,NaN,NaN
4,RSTA03,16.942114,52.380946
...,...,...,...
262,PLCR92,NaN,NaN
263,PLWI01,NaN,NaN
264,MAGA01,16.938862,52.411735
265,KATE01,16.948101,52.410016


In [12]:
slupki_przystankow.dropna()

,NAZWA,E,N
0,STAR02,16.932725,52.368768
1,BYST02,16.934045,52.370723
4,RSTA03,16.942114,52.380946
5,HEWI01,16.928839,52.383212
6,ROLN01,16.916885,52.383011
...,...,...,...
259,KATE02,16.947853,52.410274
260,MAGA02,16.938090,52.410455
264,MAGA01,16.938862,52.411735
265,KATE01,16.948101,52.410016


In [39]:
from bokeh.plotting import figure, show
from bokeh.tile_providers import get_provider, Vendors
from bokeh.models import ColumnDataSource
from bokeh.models.mappers import Mercator

import pandas as pd
import numpy as np

x_max = slupki_przystankow.max('E')
x_min = slupki_przystankow.min('E')
y_max = slupki_przystankow.max('N')
y_min = slupki_przystankow.min('N')

# Define the tile provider
tile_provider = get_provider(Vendors.CARTODBPOSITRON_RETINA)

# Define the map extents
x_range, y_range = ((x_min, x_max), 
                    (y_min, y_max)) # Web Mercator extents

# Create the Mercator projection
mercator = Mercator(
    x_range=x_range,
    y_range=y_range
)

# Convert lat/lon to Web Mercator format
x, y = mercator.map(slupki_przystankow['E'].values, slupki_przystankow['N'].values)

# Create a ColumnDataSource object
source = ColumnDataSource(data=dict(
    x=x,
    y=y,
    name=slupki_przystankow['NAZWA']
))

# Create a Bokeh figure and add the tile provider
p = figure(x_range=x_range, y_range=y_range, x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(tile_provider)

# Add the points to the figure
p.circle(x='x', y='y', size=10, source=source)

# Display the plot
show(p)


ImportError: cannot import name 'Mercator' from 'bokeh.models.mappers' (C:\Users\barto\.conda\envs\lab\lib\site-packages\bokeh\models\mappers.py)

# Zapisanie mapy do pliku CSV

In [163]:
mapa.to_csv('mapa_poznania_v1.csv', index=False)

# 